In [11]:
import parse_tfl_json
import google

In [12]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime as sjt
where tfl_message = 'ok' and
 sjt.icscode not in (select icscode from tt_h.all_stations where in_sprawl = true) and 
 sjt.icscode in (select icscode from tt_h.all_stations where lat < 53.39926695813 and lng > -2.54)
"""

df = pd.read_sql(sql, conn)

In [13]:
len(df)

5831

In [14]:
from process_journeys import get_station_lat_lng_from_icscode, get_london_icscodes, add_cycle_and_total_time, remove_journeys_not_arriving_clondon
from google import get_cycle_info
import parse_tfl_json
import json

In [35]:
for r in df.iterrows():
    index = r[0]
    row = r[1]
    
    lat, lng = get_station_lat_lng_from_icscode(row["icscode"])

    df.loc[index, "depart_lat"] = lat
    df.loc[index, "depart_lng"] = lng
    
    

    try:
        j1 = row["tfl_response"]
        j2 = json.loads(j1)
    except:
        print("parse problem")
        continue
    
    try:
        journeys = j2["journeys"]
    except:
        print("no journeys in json")
        continue
    
    # Fill in journey summary so it exists even for journeys that done end in central london
    journey = journeys[0]
    legs = journey["legs"]
    summary = parse_tfl_json.summarise_legs(legs)
    df.loc[index, "natrail_journey_summary"] = summary
    
    for journey in journeys:
        legs = journey["legs"]
        journey["legs"] = parse_tfl_json.remove_non_national_rail(legs)
    
    # Keep only journeys with a national rail component
    journeys = [j for j in journeys if len(j["legs"]) > 0]

    # Keep only journeys that end in central London  
    try:
        journeys = remove_journeys_not_arriving_clondon(journeys)
    except:
        print("Problem with journey starting at {}".format(row["station_name"]))
        continue

    
    if len(journeys) == 0:
        continue

    journeys = add_cycle_and_total_time(journeys, row["tfl_dest"])

    journeys = sorted(journeys, key = lambda x: x["total_time"])

    journey = journeys[0]
    legs = journey["legs"]

    summary = parse_tfl_json.summarise_legs(legs)
    df.loc[index, "natrail_journey_summary"] = summary

    df.loc[index, "natrail_train_changes"] = parse_tfl_json.num_changes(legs)
    df.loc[index, "final_arrival"] = parse_tfl_json.final_arrival(legs)

    travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
    df.loc[index, "natrail_journey_minutes"] = travel_time_minutes

    lat_lng = parse_tfl_json.lat_lng_from_legs(legs)
    
    
    df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
    df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]

    df.loc[index, "cycle_minutes"] = journey["cycle_minutes"]
    df.loc[index, "cycle_miles"] = journey["cycle_miles"]
    df.loc[index, "total_journeytime"] = journey["cycle_minutes"] + travel_time_minutes

    
    
cols = [c for c in df.columns if c not in ["id", "tfl_request", "tfl_response", "querydict"]]
df2 = df[cols]


In [36]:
df3 = df2.sort_values(["station_name", "tfl_dest", "total_journeytime"]).drop_duplicates(["station_name", "tfl_dest"])

In [37]:
df_pf = df3[df3["tfl_dest"] == "SW1H9AJ"]
df_cw = df3[df3["tfl_dest"] == "E145HP"]

In [38]:
all_stations_data = df_pf.merge(df_cw, how='left', on=["nlc", "icscode", "station_name", "tfl_message", "depart_lat", "depart_lng"], suffixes=("_pf", "_cw"))

In [44]:
# Write out to Postgres
all_stations_data.to_sql("stations_maps_data", engine, schema="tt_h", if_exists="replace", index=False)

In [40]:
all_stations_data.to_csv("stations_maps_data.csv", encoding='utf-8', index=False)

In [41]:
all_stations_data.rename(columns={"depart_lat":"lat", "depart_lng":"lng"}, inplace=True)
all_stations_data.to_csv("interactive_maps_template/data/stations_maps_data.csv", encoding='utf-8', index=False)

In [42]:
# What's going on with e.g. newport?
# df3[df3["icscode"] == "1000647"]


In [43]:
df_pf[df_pf.station_name.str.contains("udley")]

,nlc,icscode,station_name,tfl_message,tfl_dest,natrail_journey_summary,natrail_train_changes,final_arrival,natrail_journey_minutes,depart_lat,depart_lng,arrive_lat,arrive_lng,cycle_minutes,cycle_miles,total_journeytime
4595,7017,90000325,Audley End,ok,SW1H9AJ,Audley End Rail Station → Tottenham Hale Rail ...,NaN,NaN,NaN,52.004441,0.207160,NaN,NaN,NaN,NaN,NaN
2802,1212,90002358,Dudley Port,ok,SW1H9AJ,Dudley Port Rail Station → Birmingham New Stre...,1.0,Euston Station,113.0,52.524650,-2.049480,51.528952,-0.135,17.235056,4.021513,130.235056
2963,1149,90006655,Sandwell & Dudley,ok,SW1H9AJ,Sandwell & Dudley Rail Station → Euston Station,0.0,Euston Station,97.0,52.508658,-2.011596,51.528952,-0.135,17.235056,4.021513,114.235056
